In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ubuntu/mesolitica-tpu.json'

In [2]:
import string

char_vocabs = [''] + list(string.ascii_lowercase + string.digits) + [' ']

In [3]:
sr = 16000
maxlen = 18
maxlen_subwords = 100
minlen_text = 1
global_count = 0

In [4]:
from google.cloud import storage
import numpy as np
import six

def to_example(dictionary):
    """Helper: build tf.Example from (string -> int/float/str list) dictionary."""
    features = {}
    for (k, v) in six.iteritems(dictionary):
        if not v:
            raise ValueError('Empty generated field: %s' % str((k, v)))
        # Subtly in PY2 vs PY3, map is not scriptable in py3. As a result,
        # map objects will fail with TypeError, unless converted to a list.
        if six.PY3 and isinstance(v, map):
            v = list(v)
        if isinstance(v[0], six.integer_types) or np.issubdtype(
            type(v[0]), np.integer
        ):
            features[k] = tf.train.Feature(
                int64_list=tf.train.Int64List(value=v)
            )
        elif isinstance(v[0], float):
            features[k] = tf.train.Feature(
                float_list=tf.train.FloatList(value=v)
            )
        elif isinstance(v[0], six.string_types):
            if not six.PY2:  # Convert in python 3.
                v = [bytes(x, 'utf-8') for x in v]
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        elif isinstance(v[0], bytes):
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        else:
            raise ValueError(
                'Value for %s is not a recognized type; v: %s type: %s'
                % (k, str(v[0]), str(type(v[0])))
            )
    return tf.train.Example(features=tf.train.Features(feature=features))

In [6]:
from glob import glob
from tqdm import tqdm

files = glob('ST-CMDS-20170001_1-OS/*.txt')
len(files)

102600

In [7]:
texts = []
for f in tqdm(files):
    with open(f) as fopen:
        data = fopen.read()
        texts.append(data)

100%|██████████| 102600/102600 [00:23<00:00, 4451.23it/s]


In [8]:
files = list(zip(files, texts))

In [9]:
files[0]

('ST-CMDS-20170001_1-OS/20170001P00053I0110.txt', '你不是拐着弯的损我吧')

In [10]:
import tensorflow as tf
import malaya_speech

/home/ubuntu/.local/lib/python3.7/site-packages/malaya_boilerplate/frozen_graph.py:25: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [12]:
pinyin.get(files[0][1], format="strip", delimiter=" ")

'ni bu shi guai zhao wan de sun wo ba'

In [15]:
os.path.exists(files[0][0].replace('.txt', '.wav'))

True

In [16]:
import re

def loop(files):
    import pinyin
    
    client = storage.Client()
    bucket = client.bucket('mesolitica-tpu-general')
    files, index = files
    output_file = f'{index}-{global_count}.tfrecord'
    writer = tf.io.TFRecordWriter(output_file)
    for s in tqdm(files):
        try:
            t = pinyin.get(s[1], format="strip", delimiter=" ")
            f = s[0].replace('.txt', '.wav')
            if len(s[1]) < minlen_text:
                continue
            y, _ = malaya_speech.load(f)
            if (len(y) / sr) > maxlen:
                continue
            
            t = ''.join([c if c in char_vocabs else ' ' for c in t])
            t = re.sub(r'[ ]+', ' ', t).strip()
            
            new_t = [char_vocabs.index(c) for c in t]
            example = to_example({'waveforms': y.tolist(), 
                                  'targets': new_t, 
                                  'targets_length': [len(new_t)],
                                 'lang': [2]})
            writer.write(example.SerializeToString())
        except Exception as e:
            print(e)
            pass
    writer.close()
    blob = bucket.blob(f'mandarin-v2/{output_file}')
    blob.upload_from_filename(output_file)
    os.system(f'rm {output_file}')

In [17]:
# loop((files[:100], 0))

In [18]:
import mp

batch_size = 5000
for i in range(0, len(files), batch_size):
    batch = files[i: i + batch_size]
    mp.multiprocessing(batch, loop, cores = 8, returned = False)
    global_count += 1

100%|██████████| 325/325 [00:09<00:00, 32.97it/s]
